In [1]:
# Notebook to run the Peng 2021 sims and save the results as npy files in the respective model folders

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import cantera as ct
import concurrent.futures

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
model_dirs = glob.glob('/work/westgroup/harris.se/autoscience/fuels/butane/official/with_lib/butane_*')

In [3]:
# Load the experimental conditions
# ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
ignition_delay_data = '/home/harris.se/peng_2021.csv'
df_exp = pd.read_csv(ignition_delay_data)

N_tables = len(set(df_exp['Table'].values.astype(int)[:-1]))

In [4]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def get_delay(gas, T_orig, P_orig, X_orig):
    # function to run a RCM simulation
    
    atols = [1e-15, 1e-15, 1e-18]
    rtols = [1e-9, 1e-12, 1e-15]
    for attempt_index in range(0, len(atols)):
        T = T_orig
        P = P_orig
        X = X_orig
    

        t_end = 1.0  # time in seconds
        gas.TPX = T, P, X

        env = ct.Reservoir(ct.Solution('air.yaml'))
        reactor = ct.IdealGasReactor(gas)
        wall = ct.Wall(reactor, env, A=1.0, velocity=0)
        reactor_net = ct.ReactorNet([reactor])
        reactor_net.atol = atols[attempt_index]
        reactor_net.rtol = rtols[attempt_index]

        times = [0]
        T = [reactor.T]
        P = [reactor.thermo.P]
        X = [reactor.thermo.X]  # mol fractions
        
        MAX_STEPS = 10000
        step_count = 0
        failed = False
        while reactor_net.time < t_end:
            try:
                reactor_net.step()
            except ct._cantera.CanteraError:
                print(f'Reactor failed to solve! {attempt_index}')
                failed = True
                break

            times.append(reactor_net.time)
            T.append(reactor.T)
            P.append(reactor.thermo.P)
            X.append(reactor.thermo.X)

            step_count += 1
            if step_count > MAX_STEPS:
                print(f'Too many steps! Reactor failed to solve! {attempt_index}')
                # print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                failed = True
                break
        
        if not failed:
            slopes = np.gradient(P, times)
            delay_i = np.argmax(slopes)
            return times[delay_i]
        return -1


In [5]:
def run_simulation(condition_index):
    gas = ct.Solution(butane_yaml)
    X = concentrations[condition_index]
    delay = get_delay(gas, temperatures[condition_index], pressures[condition_index], X)
    print(f'Completed {condition_index}:\t {delay}')
    return delay

In [6]:
len(model_dirs)

3

In [7]:
model_dirs

['/work/westgroup/harris.se/autoscience/fuels/butane/official/with_lib/butane_20240401',
 '/work/westgroup/harris.se/autoscience/fuels/butane/official/with_lib/butane_20240413',
 '/work/westgroup/harris.se/autoscience/fuels/butane/official/with_lib/butane_20240418']

In [9]:
for table_index in list(set(df_exp['Table'].values.astype(int)[:-1])):
    print(f'Running sims for Table {table_index}')
    
    
    # slice just the table of interest
    table7 = df_exp[df_exp['Table'] == table_index]
    tau7 = table7['time (us)'].values.astype(float) / 1000.0  # ignition delay
    T7 = table7['T_i (K)'].values  # Temperatures
    P7 = table7['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm
    # list of starting conditions
    # Mixture compositions taken from table 2 of
    # https://www-sciencedirect-com.ezproxy.neu.edu/science/article/pii/S0010218021003898?via%3Dihub
    concentrations = []
    x_diluent = table7['%CO2'].values.astype(float)[0]
    phi = table7['phi'].values.astype(float)[0]
    x_o2 = (1.0 - x_diluent) / (1.0 + 2 / 13.0 * phi)
    x_butane = 1.0 - x_diluent - x_o2

    conc_dict = {
        'O2(2)': x_o2,
        'butane(1)': x_butane,
        'CO2(7)' : x_diluent
    }
    
    # Run simulations 10% past either end of the temperature range
    T_range = np.max(T7) - np.min(T7)
    Tmin = np.min(T7) - 0.05 * T_range
    Tmax = np.max(T7) + 0.05 * T_range
    N = 31
    temperatures = np.linspace(Tmin, Tmax, N)
    concentrations = [conc_dict] * N
    pressures = [P7[0]] * N
    
    
    for model_index in range(len(model_dirs)):
#     for model_index in [6]:
        print(f'\tRunning model {model_dirs[model_index]}')
        butane_yaml = os.path.join(model_dirs[model_index], 'chem_annotated.yaml')
        
        # Run all simulations in parallel
        delays = np.zeros(N)
        condition_indices = np.arange(0, N)
        with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
            for condition_index, delay_time in zip(condition_indices, executor.map(run_simulation, condition_indices)):
                delays[condition_index] = delay_time
                
        # save results to a table
        result_dir = os.path.join(model_dirs[model_index], 'peng_2021')
        os.makedirs(result_dir, exist_ok=True)
        result_file = os.path.join(result_dir, f'table_{table_index}.npy')
        
        results = np.concatenate((np.matrix(temperatures), np.matrix(delays)))
        np.save(result_file, results)
        

Running sims for Table 1
	Running model /work/westgroup/harris.se/autoscience/fuels/butane/official/with_lib/butane_20240401
Completed 9:	 0.00048391671673936863
Completed 6:	 0.0005902554814621339Completed 7:	 0.0005520565824168877

Completed 10:	 0.0004534250371528019Completed 0:	 0.0008924557722451119

Completed 15:	 0.0003307010111958558
Completed 13:	 0.00037452613573426283
Completed 14:	 0.0003518472697459156
Completed 3:	 0.0007238910297621104Completed 4:	 0.000675853590740778
Completed 12:	 0.00039892864130590936

Completed 11:	 0.0004251677210575038Completed 8:	 0.0005166843262746768
Completed 2:	 0.0007757449119584037

Completed 1:	 0.0008318456978625914
Completed 5:	 0.0006313932842515216
Completed 30:	 0.00014055790569520167
Completed 16:	 0.00031107149067608766
Completed 29:	 0.00014821242773642687Completed 28:	 0.00015635172216622585

Completed 22:	 0.00021819428524493038
Completed 25:	 0.00018424838875354842
Completed 18:	 0.0002757232376596891
Completed 27:	 0.000165023

Completed 18:	 0.00013468015497663128
Completed 19:	 0.00012493893257282688
Completed 21:	 0.00010796002225133027
Completed 20:	 0.00011606978936203566
Completed 16:	 0.00015715223381720657
Completed 26:	 7.661580156735107e-05
Completed 17:	 0.00014537696124116944
Completed 23:	 9.378573107172402e-05
Completed 24:	 8.756610624037448e-05
Completed 29:	 6.325471630988543e-05
Completed 25:	 8.186794895869063e-05
Completed 22:	 0.00010054581832103503
Completed 27:	 7.180263905705256e-05
Completed 30:	 5.9465639501817265e-05
Completed 28:	 6.73528515614369e-05
	Running model /work/westgroup/harris.se/autoscience/fuels/butane/official/with_lib/butane_20240413
Completed 14:	 0.0002489838860136244
Completed 15:	 0.00022830764446898262
Completed 6:	 0.0005192774559398149
Completed 4:	 0.0006309567279747804
Completed 7:	 0.00047183164444297184
Completed 11:	 0.00032516355375789387
Completed 8:	 0.00042916142395925454Completed 5:	 0.0005721117870644765
Completed 2:	 0.0007696891911993179

Complet

Completed 11:	 0.000502293560263023
Completed 4:	 0.0007187738257626134
Completed 2:	 0.0007981344064797605Completed 1:	 0.0008413254056741722

Completed 9:	 0.0005556601613004308Completed 0:	 0.0008870151934957948

Completed 8:	 0.0005846761945631562Completed 10:	 0.0005282024062565686

Completed 5:	 0.000682375678736022
Completed 6:	 0.0006479175351543563
Completed 7:	 0.00061540654232647
Completed 15:	 0.00041188024067134873
Completed 12:	 0.0004777561991808151
Completed 13:	 0.00045454371350075535
Completed 14:	 0.00043261505702176943
Completed 28:	 0.0002249264833877619
Completed 30:	 0.00020611181130775013
Completed 27:	 0.00023506380708886198
Completed 29:	 0.0002152497479714654
Completed 26:	 0.0002458135328691051
Completed 25:	 0.0002571174605013805
Completed 23:	 0.00028170300278155234
Completed 22:	 0.00029500785323753284
Completed 24:	 0.00026905880797152483
Completed 16:	 0.0003922869522359516Completed 18:	 0.0003562618436538041

Completed 17:	 0.0003737749383341127
Comple

In [ ]:
np.concatenate((np.matrix(iter0), np.matrix(iter5))).shape

In [ ]:
P7

In [ ]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def get_delay(gas, T_orig, P_orig, X_orig):
    # function to run a RCM simulation
    
    atols = [1e-15, 1e-15, 1e-18]
    rtols = [1e-9, 1e-12, 1e-15]
    for attempt_index in range(0, len(atols)):
        T = T_orig
        P = P_orig
        X = X_orig
    

        t_end = 1.0  # time in seconds
        gas.TPX = T, P, X

        env = ct.Reservoir(ct.Solution('air.yaml'))
        reactor = ct.IdealGasReactor(gas)
        wall = ct.Wall(reactor, env, A=1.0, velocity=0)
        reactor_net = ct.ReactorNet([reactor])
        reactor_net.atol = atols[attempt_index]
        reactor_net.rtol = rtols[attempt_index]

        times = [0]
        T = [reactor.T]
        P = [reactor.thermo.P]
        X = [reactor.thermo.X]  # mol fractions
        
        MAX_STEPS = 10000
        step_count = 0
        failed = False
        while reactor_net.time < t_end:
            try:
                reactor_net.step()
            except ct._cantera.CanteraError:
                print(f'Reactor failed to solve! {attempt_index}')
                failed = True
                break

            times.append(reactor_net.time)
            T.append(reactor.T)
            P.append(reactor.thermo.P)
            X.append(reactor.thermo.X)

            step_count += 1
            if step_count > MAX_STEPS:
                print(f'Too many steps! Reactor failed to solve! {attempt_index}')
                # print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                failed = True
                break
        
        if not failed:
            slopes = np.gradient(P, times)
            delay_i = np.argmax(slopes)
            return times[delay_i]
        return -1
        
#         slopes = np.gradient(P, times)
#         i = np.argmax(slopes)
#         return times[i]

In [ ]:
def run_simulation(condition_index):
    gas = ct.Solution(butane_yaml)
    X = concentrations[condition_index]
    delay = get_delay(gas, T7[condition_index], P7[condition_index], X)
    print(f'Completed {condition_index}:\t {delay}')
    return delay

In [ ]:
# Run all simulations in parallel
delays = np.zeros(len(table7))
condition_indices = np.arange(0, len(table7))
with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
    for condition_index, delay_time in zip(condition_indices, executor.map(run_simulation, condition_indices)):
        delays[condition_index] = delay_time

In [ ]:
delays

In [ ]:
iter0 = np.array([0.00099921, 0.00017139, 0.00030184, 0.00039824, 0.00066331])
iter5 = np.array([0.00080206, 0.00012514, 0.00021517, 0.00028614, 0.00049938])

In [ ]:
# plot the ignition delay
# plt.plot(1000.0 / T7, full_base_rmg7, marker='x', label='large_seed_1week')
# plt.plot(1000.0 / T7, aramco7, marker='x', label='aramco')
plt.scatter(1000.0 / T7, iter0, marker='o', label='Run 0')
# plt.plot(1000.0 / T7, iter1, marker='o', label='Run 1')
# plt.plot(1000.0 / T7, iter2, marker='o', label='Run 2')
# plt.plot(1000.0 / T7, iter3, marker='o', label='Run 3')
# plt.plot(1000.0 / T7, iter4, marker='o', label='Run 4')
plt.scatter(1000.0 / T7, iter5, marker='o', label='Run 5')
# plt.plot(1000.0 / T7, iter3p2, marker='o', label='Run 3.2')
plt.scatter(1000.0 / T7, tau7 / 1000.0, color='black', label='experiment')

ax = plt.gca()
ax.set_yscale('log')
plt.legend()
# plt.legend(['Mechanism 174', 'Base RMG', 'Aramco', 'Experiment'])
# plt.legend(['RMG 24', 'RMG 1 week', 'Aramco', 'new calculation', 'Experiment'], loc=(1.04, 0))
plt.title('Ignition Delays Phi=1.0')
plt.xlabel('1000K / T')
plt.ylabel('Delay (s)')
plt.savefig('table7.png')
plt.legend(bbox_to_anchor=(1, 1))